In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
stations = ['FOL', 'AFO', 'AWB', 'AWP']
frames = {}

In [3]:
for i in stations:
    station = pd.read_csv(os.path.join('S:/DWR/VOL1/BayDeltaDRV/Tributaries/American River/American River Temperature Compilation/Time Series Data/'+i+'.csv'), 
                          parse_dates=True, index_col=0, usecols=[1,2])
    rolling = round(station.rolling(7).mean().groupby(station.index.month).mean(),1)
    rollingMax=round(station.rolling(7).mean().groupby(station.index.month).max(), 1)
    rolling.index.name = None
    rolling.rename(index={1 :'January',
                      2 :'February',
                      3 : 'March',
                      4 : 'April',
                      5 : 'May',
                      6 : 'June',
                      7 : 'July',
                      8 : 'August',
                      9 : 'September',
                      10 : 'October', 
                      11 : 'November',
                      12 : 'December'},
                      columns = {"Maximum" : "Monthly average of 7DADM"},
                      inplace=True)
    rollingMax.index.name = None
    rollingMax.rename(index={1 :'January',
                      2 :'February',
                      3 : 'March',
                      4 : 'April',
                      5 : 'May',
                      6 : 'June',
                      7 : 'July',
                      8 : 'August',
                      9 : 'September',
                      10 : 'October', 
                      11 : 'November',
                      12 : 'December'},
                      columns = {"Maximum" : "Max 7DADM"},
                      inplace=True)
    gb = station.rolling(7).mean().groupby(station.index.month)
    groups=dict(list(gb))
    percentages = []
    for j in range(1,13):
        percentages.append(int(groups[j][(groups[j]<=64)].count()/(groups[j].count())*100))
    percentage_table = pd.DataFrame(percentages, index=['January', 
                                                    'February',
                                                    'March', 
                                                    'April',
                                                    'May', 
                                                    'June', 
                                                    'July',
                                                    'August', 
                                                    'September', 
                                                    'October',
                                                    'November', 
                                                    'December'], 
                                                    columns=['Max'])
    percentage_table.rename(columns={'Max' : '% days within month that EPA criteria met'}, 
                        inplace=True)
    table = pd.concat([rollingMax, rolling, percentage_table], axis=1)
    frames[i] = table

In [4]:
# Example cell from Marysville
station
#rolling
# rollingMax
# percentage_table

,Maximum
Date,
2000-10-11,61.70
2000-10-12,64.40
2000-10-13,64.40
2000-10-14,65.30
2000-10-15,66.20
2000-10-16,66.20
2000-10-17,66.20
2000-10-18,66.20
2000-10-19,66.20


In [5]:
frames['AFO']

,Max 7DADM,Monthly average of 7DADM,% days within month that EPA criteria met
January,54.0,48.6,100
February,53.3,48.3,100
March,58.3,50.6,100
April,63.9,54.0,100
May,64.1,56.9,99
June,66.9,59.2,95
July,70.4,62.5,72
August,71.9,64.6,45
September,72.8,65.0,40
October,70.9,63.1,62


In [6]:
with pd.ExcelWriter('S:/DWR/VOL1/BayDeltaDRV/Tributaries/American River/American River Temperature Compilation/Data Tables/DADM_tables.xlsx') as writer:
    frames['FOL'].to_excel(writer, sheet_name='Below Folsom Dam')
    frames['AFO'].to_excel(writer, sheet_name='At Fair Oaks Blvd')
    frames['AWB'].to_excel(writer, sheet_name='At Watt Avenue Bridge')
    frames['AWP'].to_excel(writer, sheet_name='At William B Pond Park')